In [40]:
"""
dividend_preprocess.py
────────────────────────────────────────────────────────
배당 공시 원본 CSV → ML 학습용 CSV 전처리 파이프라인
 - 불필요 컬럼 제거
 - 핵심 수치형 결측 처리 (제거 or 보정)
 - 숫자 클렌징
 - 지급일 파생변수 생성 (days_to_payment)
 - 중복 제거 + 기업별 공시 건수 저장
────────────────────────────────────────────────────────
"""

import pandas as pd
from pathlib import Path

# ─────────────────────────────────────────────────────
# 설정값
# ─────────────────────────────────────────────────────
INPUT_PATH   = Path("data/dividend_final_for_ml.csv")
OUTPUT_PATH  = Path("data/dividend_ml_ready.csv")

DROP_COLS    = ["div_type", "div_kind", "per_share_preferred", "yield_preferred"]
NUM_COLS     = ["per_share_common", "yield_common", "total_amount"]
DEDUP_KEYS   = ["corp_name", "rcept_dt"]

# ─────────────────────────────────────────────────────
# 유틸 함수
# ─────────────────────────────────────────────────────
def to_float_series(s: pd.Series) -> pd.Series:
    """콤마·대시 제거 후 float 변환 (실패 시 NaN)."""
    return (
        s.astype(str)
         .str.replace(",", "", regex=False)
         .str.replace("-", "", regex=False)
         .str.strip()
         .pipe(pd.to_numeric, errors="coerce")
    )

# ─────────────────────────────────────────────────────
# 메인 전처리 함수
# ─────────────────────────────────────────────────────
def preprocess_dividend_csv(
    input_path: Path = INPUT_PATH,
    output_path: Path = OUTPUT_PATH,
    firm_count_path: Path = FIRM_COUNTS
) -> pd.DataFrame:
    # 1) 원본 로드
    df = pd.read_csv(input_path, encoding="utf-8-sig")
    print(f"📄 원본 shape : {df.shape}")

    # 2) 불필요 컬럼 제거
    df = df.drop(columns=[c for c in DROP_COLS if c in df.columns])

    # 3) 숫자형 컬럼 전처리
    for c in NUM_COLS:
        if c in df.columns:
            df[c] = to_float_series(df[c])

    # 4) 수치형 결측 처리
    if "total_amount" in df.columns:
        df["total_amount"] = df["total_amount"].fillna(df["total_amount"].median())

    if "yield_common" in df.columns:
        df["yield_common"] = df["yield_common"].fillna(df["yield_common"].median())

    # 5) 핵심 per_share_common 결측 제거
    df = df.dropna(subset=["per_share_common"])
    print(f"🧹 per_share_common 결측 제거 후: {df.shape}")

    # 6) 날짜 처리: '-' → NaT, 날짜형 변환
    df["rcept_dt"] = pd.to_datetime(df["rcept_dt"], errors="coerce")
    df["payment_date"] = df["payment_date"].replace("-", pd.NA)
    df["payment_date"] = pd.to_datetime(df["payment_date"], errors="coerce")

    # 7) 파생 변수: days_to_payment (+ 결측 마스킹)
    df["days_to_payment"] = (df["payment_date"] - df["rcept_dt"]).dt.days
    df["days_to_payment_missing"] = df["days_to_payment"].isna().astype(int)
    df["days_to_payment"] = df["days_to_payment"].fillna(-1)  # 또는 -999도 가능

    # 8) 중복 제거
    before = df.shape[0]
    df = df.drop_duplicates(subset=DEDUP_KEYS)
    print(f"🧹 중복 제거: {before} → {df.shape[0]}행")

    # 9) 최종 저장
    output_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"💾 최종 데이터 저장 완료 → {output_path.resolve()} (shape: {df.shape})")

    return df, firm_counts

# ─────────────────────────────────────────────────────
# 실행
# ─────────────────────────────────────────────────────
if __name__ == "__main__":
    df_result, firm_counts = preprocess_dividend_csv()

    print("\n🔍 미리보기 (상위 5개 샘플):")
    print(df_result.head().to_string(index=False))

    print("\n🏆 공시 건수 Top 10:")
    print(firm_counts.head(10).to_string(index=False))

📄 원본 shape : (27232, 17)
🧹 per_share_common 결측 제거 후: (22321, 13)
🧹 중복 제거: 22321 → 17568행
💾 최종 데이터 저장 완료 → /Users/gun/Desktop/미래에셋 AI 공모전/data/dividend_ml_ready.csv (shape: (17568, 15))

🔍 미리보기 (상위 5개 샘플):
        corp_name  stock_code                      rcept_dt              report_nm       rcept_no  per_share_common  yield_common  total_amount record_date payment_date meeting_held meeting_date board_decision_date  days_to_payment  days_to_payment_missing
            유니온스틸        3640 1970-01-01 00:00:00.020130207              현금ㆍ현물배당결정 20130207800442             100.0           0.9  1.025968e+09  2012-12-31          NaT           개최            -          2013-02-07             -1.0                        1
            유니온스틸        3640 1970-01-01 00:00:00.020140206              현금ㆍ현물배당결정 20140206800401             300.0           2.0  3.077903e+09  2013-12-31          NaT           개최            -          2014-02-06             -1.0                        1
              